## SHUBHAM SHARMA
### IIT BOMBAY
This code is for the implimentation of 3D U-Net

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#Importing libraries
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

#######################
#For archtecture
from keras.layers import Input,Conv3D,MaxPooling3D,BatchNormalization,Conv3DTranspose,concatenate,Reshape,Permute,Activation
from keras.models import Model
import keras.backend as K
from keras.callbacks import ModelCheckpoint
from keras.optimizers import adam

#######################
#snippit
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
#from FirstPackage.Network import net_1
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.85
set_session(tf.Session(config=config))

Using TensorFlow backend.


##### Let us first define the architecture of the network

In [0]:
K.set_image_data_format('channels_first')#For channel first

def U_Net(length,width, height, depth, classes=2,weightsPath=None):
    '''
    Considering the inputs to be channel first
    if the imgae is (1,512,512,120)
    then length=512,width=512,height=120,depth=1 , depth tells about the different channels
    '''
    data_shape = length*width*height
    bn_flag=True #Make this flag False in case of testing
    
    inputs = Input((depth,length,width, height))
    conv1 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(inputs)
    conv1 = BatchNormalization(axis=1)(conv1,training=bn_flag)
    conv1 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization(axis=1)(conv1,training=bn_flag)
    pool1 = MaxPooling3D(pool_size=(2, 2, 2))(conv1)
    
    
    
#     conv2 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(pool1)
#     conv2 = BatchNormalization(axis=1)(conv2,training=bn_flag)
#     conv2 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(conv2)
#     conv2 = BatchNormalization(axis=1)(conv2,training=bn_flag)
#     pool2 = MaxPooling3D(pool_size=(2, 2, 2))(conv2)

#     conv3 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(pool2)
#     conv3 = BatchNormalization(axis=1)(conv3,training=bn_flag)
#     conv3 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(conv3)
#     conv3 = BatchNormalization(axis=1)(conv3,training=bn_flag)
#     pool3 = MaxPooling3D(pool_size=(2, 2, 2))(conv3)
    
#     conv4 = Conv3D(256, (3, 3, 3), activation='relu', padding='same')(pool3)
#     conv4 = BatchNormalization(axis=1)(conv4,training=bn_flag)
#     conv4 = Conv3D(256, (3, 3, 3), activation='relu', padding='same')(conv4)
#     conv4 = BatchNormalization(axis=1)(conv4,training=bn_flag)
#     pool4 = MaxPooling3D(pool_size=(2, 2, 2))(conv4)
    
#     conv5 = Conv3D(512, (3, 3, 3), activation='relu', padding='same')(pool4)
#     conv5 = Conv3D(512, (3, 3, 3), activation='relu', padding='same')(conv5)

    conv5 = Conv3D(512, (3, 3, 3), activation='relu', padding='same')(pool1)
    conv5 = Conv3D(512, (3, 3, 3), activation='relu', padding='same')(conv5)

#     up6 = concatenate([Conv3DTranspose(256, (2, 2, 2), strides=(2, 2, 2), padding='same')(conv5), conv4], axis=1)
#     conv6 = Conv3D(256, (3, 3, 3), activation='relu', padding='same')(up6)
#     conv6 = Conv3D(256, (3, 3, 3), activation='relu', padding='same')(conv6)
    
#     up7 = concatenate([Conv3DTranspose(128, (2, 2, 2), strides=(2, 2, 2), padding='same')(conv6), conv3], axis=1)
#     conv7 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(up7)
#     conv7 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(conv7)
    
#     up8 = concatenate([Conv3DTranspose(64, (2, 2, 2), strides=(2, 2, 2), padding='same')(conv7), conv2], axis=1)
#     conv8 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(up8)
#     conv8 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(conv8)
    
    up9 = concatenate([Conv3DTranspose(32, (2, 2, 2), strides=(2, 2, 2), padding='same')(conv8), conv1], axis=1)
    conv9 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(conv9)
    
    conv10 = Conv3D(classes, (3, 3, 3), padding='same')(conv9)
          
    reshape = Reshape((classes, data_shape), input_shape=(classes,length, width, height))(conv10)
    perm = Permute((2, 1))(reshape)
    softmax = Activation("softmax")(perm)
    
    model = Model(inputs=[inputs], outputs=[softmax])
    
    if weightsPath is not None:
        model.load_weights(weightsPath)
    
    return model 


## Cardiac dataset

In [0]:
X_cardiac=np.load('gdrive/My Drive/Phase 1 Project/X_cardiac.npy')
Y_cardiac=np.load('gdrive/My Drive/Phase 1 Project/Y_cardiac_categorical.npy')
X_cardiac=X_cardiac#.astype('float32')
Y_cardiac=Y_cardiac#.astype('float32')
X_cardiac_whitened = np.zeros(X_cardiac.shape)
for i in range(X_cardiac.shape[0]):
  temp1=X_cardiac[i]
  temp2=(temp1 - np.mean(temp1))/(np.std(temp1))
  X_cardiac_whitened[i]=temp2

In [0]:
X_cardiac_whitened=np.reshape(X_cardiac_whitened,(X_cardiac_whitened.shape[0],  1 ,X_cardiac_whitened.shape[1] ,X_cardiac_whitened.shape[2] ,X_cardiac_whitened.shape[3]))

In [0]:
x_train_cardiac, x_test_cardiac, y_train_cardiac, y_test_cardiac = train_test_split(X_cardiac_whitened, Y_cardiac, test_size=0.20)
print('The shape of x_train_cardiac is :',x_train_cardiac.shape)
print('The shape of x_test_cardiac is :',x_test_cardiac.shape)
print('The shape of y_train_cardiac is :',y_train_cardiac.shape)
print('The shape of y_test_cardiac is :',y_test_cardiac.shape)

The shape of x_train_cardiac is : (16, 1, 320, 320, 128)
The shape of x_test_cardiac is : (4, 1, 320, 320, 128)
The shape of y_train_cardiac is : (16, 13107200, 2)
The shape of y_test_cardiac is : (4, 13107200, 2)


In [0]:
length,width,height,classes,depth=320,320,128,2,1
weightsPath=None
cardiac = U_Net(length,width, height, depth, classes=2,weightsPath=None)
print(cardiac.summary())

# path2save='./cardiac.{epoch:02d}-{loss:.2f}-{val_loss:.2f}.hdf5'
modelCheck = ModelCheckpoint('/.cardiac.{epoch:04d}-{acc:.4f}-{loss:.4f}.h5', monitor='acc', mode='auto')

opt = adam(lr=1e-4)
#Set the compiler parameter for the training
cardiac.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"],sample_weight_mode='auto')
print ("Training the Model...")


NameError: ignored

In [0]:

#Train the Network
cardiac.fit(x_train_cardiac, y_train_cardiac, batch_size = 1, epochs= 50, verbose=0,callbacks= [modelCheck],validation_data=(x_test_cardiac, y_test_cardiac))
